In [1]:
import os

In [2]:
test_image_dir="../data/test_images_phase1/"
test_csv="../data/test_phase1_v2.csv"

In [3]:
import pandas as pd

submission_df = pd.read_csv(test_csv)
submission_df.head()

,img_fName,img_w,img_h
0,ce8f0ede-4366-45b9-bdd4-959e17c7618c.jpeg,1599,965
1,f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg,822,861
2,99c6b7f9-dbea-4635-a18d-cde16b125515.jpeg,720,1280
3,662b4405-3600-41b9-9812-451b6d9df322.jpeg,3024,4032
4,1de14b6b-6781-4a24-ae5e-f760e865c758.jpeg,1242,2208


In [4]:
from hydra import compose, initialize, core as hydra_core

hydra_core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="../exps", version_base=None)
cfg = compose(config_name="submission")


In [5]:
import sys

sys.path.append("../")

In [6]:
import torch
from mosquito.models import models

model = models[cfg.model_name](cfg, num_classes=6)
model.load_state_dict(torch.load(cfg.model_path, map_location=torch.device("cpu")))

<All keys matched successfully>

In [7]:
test_images = [os.path.join(test_image_dir, x) for x in submission_df["img_fName"].values]
test_ids = list(range(len(test_images)))

In [14]:
import PIL.Image
from tqdm.notebook import tqdm_notebook
import albumentations as A
import numpy as np
import albumentations.pytorch

from mosquito.datasets import datasets

bboxes_preds = []
labels_preds = []

model.eval()

dataset_cls = datasets[cfg.dataset_name]

for image_id, img  in tqdm_notebook(list(zip(test_ids, test_images))):
    with torch.no_grad():
        image = np.array(PIL.Image.open(img).convert("RGB"))
        imgs = A.Compose([
            # A.Resize(width=cfg.input_size, height=cfg.input_size),
            albumentations.pytorch.ToTensorV2()
        ])(image=image)["image"].float().unsqueeze(0)  / 255.
        outputs = model.model(imgs)
        outputs = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in outputs]
        outputs = {0: outputs[0]}
        res = model.postprocess(outputs)
        
        # append results
        bboxes_preds.append(res[0]["boxes"].numpy())
        labels_preds.append(dataset_cls.integer_to_word[res[0]["labels"].item()])

  0%|          | 0/2676 [00:00<?, ?it/s]

In [15]:
predictions = [list(bboxes_preds[i][0]) + [labels_preds[i]] for i in range(len(test_images))]
predictions_df = pd.DataFrame(predictions, columns=["bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr", "class_label"])

submission_df = pd.concat([submission_df, predictions_df], axis=1)
submission_df.drop(columns=["img_w", "img_h"], inplace=True)

submission_df.head()

,img_fName,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr,class_label
0,ce8f0ede-4366-45b9-bdd4-959e17c7618c.jpeg,550.203064,275.218811,1164.874512,762.542603,culex
1,f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg,19.554203,53.384151,750.967712,859.897278,culex
2,99c6b7f9-dbea-4635-a18d-cde16b125515.jpeg,259.021973,571.430786,558.624817,807.684326,albopictus
3,662b4405-3600-41b9-9812-451b6d9df322.jpeg,1677.877319,1600.402832,2234.702637,2227.840576,culex
4,1de14b6b-6781-4a24-ae5e-f760e865c758.jpeg,400.784973,750.484192,832.935730,1056.553833,albopictus


In [16]:
submission_df.to_csv("../submissions/submission_baseline.csv", index=False)

In [17]:
!aicrowd submission create -c mosquitoalert-challenge-2023 -f "../submissions/submission_baseline.csv"

An unexpected error occured!
cannot import name 'appengine' from 'urllib3.contrib' (/mnt/data/home/manuel/anaconda3/envs/mos/lib/python3.9/site-packages/urllib3/contrib/__init__.py)
To get more information, you can run this command with -v.
To increase level of verbosity, you can go upto -vvvvv
